# Setup

In [1]:
from langgraph.graph import StateGraph
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
#from duckduckgo_search import DDGS
from langchain_community.tools import GoogleSerperRun
from langchain_community.utilities import GoogleSerperAPIWrapper
from typing import TypedDict
import graphviz
import os
import yaml

In [2]:
with open("config.yaml", "r") as file:
    config = yaml.safe_load(file)
os.environ["OPENAI_API_KEY"] = config["OPENAI_API_KEY"]
os.environ['SERPER_API_KEY'] = config['SERPER_API_KEY']
llm = ChatOpenAI(model_name="gpt-4.1-nano", temperature=0.3)

# Estado

In [3]:
class State(TypedDict):
    pergunta: str  
    conteudo: str 
    resposta: str 

# Funções de CallBack

In [27]:
def recebe_pergunta(state: State) -> State:
    print(f"Usuário perguntou: {state['pergunta']}")
    return {"pergunta": state["pergunta"]}

def precisa_pesquisar(state: State) -> State:
    pergunta = state["pergunta"].lower()
    precisa = any(p in pergunta for p in ["dados", "estatísticas", "números", "pesquisa"])
    print("Precisa pesquisar?", precisa)
    # Retorna um dicionário, com chave especial para decisão
    return {"next_step": "pesquisar" if precisa else "consultar_llm"}

def pesquisar(state: State) -> State:
    pergunta = state["pergunta"]

    api_wrapper = GoogleSerperAPIWrapper(
        gl="br",
        hl="pt-br",
    )

    google_search = GoogleSerperRun(api_wrapper=api_wrapper, k=10, gl="br", hl="pt-br", search_type="news")
    print(f"Pesquisando no Google: {pergunta}")

    resultados = google_search.run(pergunta)
    print(f"Resultados da pesquisa: {resultados}")

    if resultados:
        contexto = resultados
    else:
        contexto = "Nenhum resultado encontrado."

    print("Contexto obtido da pesquisa.")
    print(contexto)

    return {"conteudo": contexto}    

def consultar_llm(state: State) -> State:
    prompt = ChatPromptTemplate.from_template("Responda à seguinte pergunta: {pergunta}")
    chain = prompt | llm
    resposta = chain.invoke({"pergunta": state["pergunta"]})
    print("Resposta direta do LLM.")
    return {"resposta": resposta.content}

def sintetizar(state: State) -> State:
    contexto = state.get("conteudo", "")
    print("Contexto para síntese:", contexto)
    pergunta = state["pergunta"]
    print("Pergunta para síntese:", pergunta)
    prompt = ChatPromptTemplate.from_template("""
    Use o seguinte contexto para responder a pergunta:
    Contexto: {contexto}
    Pergunta: {pergunta}
    Resposta:""")
    chain = prompt | llm
    resposta = chain.invoke({"contexto": contexto, "pergunta": pergunta})
    print("Resposta sintetizada com contexto.")
    return {"resposta": resposta.content}

def responder(state: State) -> State:
    print("\n Resposta Final:")
    print(state["resposta"])
    return state

# Cria Grafo

In [28]:
graph = StateGraph(State)

graph.add_node("recebe_pergunta", RunnableLambda(recebe_pergunta))
graph.add_node("decisao", RunnableLambda(precisa_pesquisar))
graph.add_node("pesquisar", RunnableLambda(pesquisar))
graph.add_node("consultar_llm", RunnableLambda(consultar_llm))
graph.add_node("sintetizar", RunnableLambda(sintetizar))
graph.add_node("responder", RunnableLambda(responder))

# Transições de Estado e Condições

In [29]:
graph.set_entry_point("recebe_pergunta")

graph.add_edge("recebe_pergunta", "decisao")
graph.add_conditional_edges(
    "decisao",
    lambda state: state["next_step"],  
    {
        "pesquisar": "pesquisar",
        "consultar_llm": "consultar_llm"
    }
)
graph.add_edge("pesquisar", "sintetizar")
graph.add_edge("consultar_llm", "responder")
graph.add_edge("sintetizar", "responder")
graph.set_finish_point("responder")

# Execução

In [30]:
executable = graph.compile()

print("\n TESTE 1:")
executable.invoke({"pergunta": "Qual é a capital da Alemanha?"})

print("\n  TESTE 2:")
executable.invoke({"pergunta": "Me mostre dados sobre economia brasileira em 2025."})


 TESTE 1:
Usuário perguntou: Qual é a capital da Alemanha?
Precisa pesquisar? False
Resposta direta do LLM.

 Resposta Final:
A capital da Alemanha é Berlim.

  TESTE 2:
Usuário perguntou: Me mostre dados sobre economia brasileira em 2025.
Precisa pesquisar? True
Pesquisando no Google: Me mostre dados sobre economia brasileira em 2025.
Resultados da pesquisa: Como já esperado pelos analistas, a atividade econômica perdeu força em 2025 após avanço de 3,4% em 2024 — o terceiro ano seguido de crescimento ... A economia brasileira cresceu 0,1% no terceiro trimestre de 2025 em comparação com o segundo trimestre e acumula avanço de 2,5% no período ... Nesse cenário, a SPE passou a projetar crescimento de 2,2% para a economia brasileira em 2025. “Uma revisão marginal, que reflete aumento da ... Projeção para o Brasil é de um crescimento de 2,4% em 2025, quinto maior entre os países do G20. Acesse na íntegra no Bora Investir. O Instituto de Pesquisa Econômica Aplicada (Ipea) manteve a previsã

{'pergunta': 'Me mostre dados sobre economia brasileira em 2025.',
 'conteudo': 'Como já esperado pelos analistas, a atividade econômica perdeu força em 2025 após avanço de 3,4% em 2024 — o terceiro ano seguido de crescimento ... A economia brasileira cresceu 0,1% no terceiro trimestre de 2025 em comparação com o segundo trimestre e acumula avanço de 2,5% no período ... Nesse cenário, a SPE passou a projetar crescimento de 2,2% para a economia brasileira em 2025. “Uma revisão marginal, que reflete aumento da ... Projeção para o Brasil é de um crescimento de 2,4% em 2025, quinto maior entre os países do G20. Acesse na íntegra no Bora Investir. O Instituto de Pesquisa Econômica Aplicada (Ipea) manteve a previsão de crescimento da economia brasileira em 2,4% para 2025. Para 2026, a ... No acumulado do primeiro semestre de 2025, o crescimento foi de 2,5%, e nos últimos quatro trimestres, a economia expandiu-se em 3,2%. Do lado ... O Produto Interno Bruto (PIB) do Brasil variou 0,1% no terc

# Gráfico

In [31]:
dot = executable.get_graph().draw_mermaid()

print(dot)

%%{init: {'flowchart': {'curve': 'linear'}}}%%
graph TD;
	__start__([<p>__start__</p>]):::first
	recebe_pergunta(recebe_pergunta)
	decisao(decisao)
	pesquisar(pesquisar)
	consultar_llm(consultar_llm)
	sintetizar(sintetizar)
	responder(responder)
	__end__([<p>__end__</p>]):::last
	__start__ --> recebe_pergunta;
	consultar_llm --> responder;
	decisao -.-> consultar_llm;
	decisao -.-> pesquisar;
	pesquisar --> sintetizar;
	recebe_pergunta --> decisao;
	sintetizar --> responder;
	responder --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc

